In [ ]:
import os
import datetime
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from torchvision import models
from tqdm import tqdm
import sys
from sklearn.metrics import confusion_matrix, classification_report
from collections import Counter
from PIL import Image

In [ ]:
DATA_DIR = os.path.expanduser("~/Zoidberg/data/raw/chest_Xray/train")
MODEL_PATH = os.path.expanduser("~/Zoidberg/models/mobilenet_pneumonia.pt")
DOCS_DIR = os.path.expanduser("~/Zoidberg/docs")
TIMESTAMP = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
BATCH_SIZE = 32
EPOCHS = 10
LEARNING_RATE = 1e-4
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

def load_dataset():
    class_map = {"NORMAL": 0, "bacterial": 1, "viral": 2}
    dataset = datasets.ImageFolder(DATA_DIR, transform=data_transforms)
    new_samples = []
    for path, _ in dataset.samples:
        fname = os.path.basename(path).lower()
        if "pneumonia" in fname:
            label = class_map["bacterial"] if "bacteria" in fname else class_map["viral"]
        else:
            label = class_map["NORMAL"]
        new_samples.append((path, label))
    dataset.samples = new_samples
    dataset.targets = [s[1] for s in new_samples]
    dataset.classes = ["NORMAL", "bacterial", "viral"]
    return dataset

dataset = load_dataset()

from torch.utils.data import random_split
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE)

In [ ]:
train_losses = []
train_accuracies = []

def train():
    model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.DEFAULT)
    model.classifier[1] = nn.Linear(model.last_channel, 3)
    model = model.to(DEVICE)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    for epoch in range(EPOCHS):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}", leave=True, file=sys.stdout)
        for inputs, labels in loop:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            loop.set_postfix(loss=running_loss / (total / BATCH_SIZE), acc=correct / total)
        train_losses.append(running_loss / len(train_loader))
        train_accuracies.append(correct / total)
        print(f"Epoch {epoch+1} - Loss: {train_losses[-1]:.4f} - Accuracy: {train_accuracies[-1]:.4f}")

    torch.save(model.state_dict(), MODEL_PATH)
    print(f"Modèle sauvegardé dans : {MODEL_PATH}")
    return model

model = train()

In [ ]:
model.eval()
y_true, y_pred = [], []
class_indices = [0, 1, 2]
with torch.no_grad():
    for inputs, labels_batch in tqdm(val_loader, desc="Évaluation", file=sys.stdout):
        inputs = inputs.to(DEVICE)
        labels_batch = labels_batch.to(DEVICE)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        y_true.extend(labels_batch.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

print("Répartition réelle dans y_true :", Counter(y_true))
print("Répartition prédite dans y_pred :", Counter(y_pred))

cm = confusion_matrix(y_true, y_pred, labels=class_indices)
report = classification_report(y_true, y_pred, labels=class_indices, target_names=dataset.classes, zero_division=0)
print(report)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=dataset.classes, yticklabels=dataset.classes)
plt.title(f"Matrice de confusion ({TIMESTAMP})")
plt.xlabel("Prédit")
plt.ylabel("Réel")
plt.tight_layout()
plt.show()